# 05 Feature space

## TODO

- Sæt labels på x og y aksen

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import fasttext

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D

from tabulate import tabulate

In [4]:
data = pd.read_csv('./data/sklearn.csv')

model = fasttext.load_model('./data/fasttext_trained_model.bin')
data['vec'] = data['review'].apply(lambda x: model.get_sentence_vector(x))

In [5]:
def nearest_neighbours(*args):
    numbers_of_args= len(args)
    listDict = {}   
    
    words_lists = [[] for i in range(1, numbers_of_args +1 )]
    
    for i in range(1, numbers_of_args+1 ):
        listDict["Title_"+str(i)] = []
    
    count = 1
    while count < numbers_of_args+1:
       
        for a in args[count-1:count]:
            listDict["Title_"+str(count)].append(model.get_nearest_neighbors(a))
            count += 1 
            break
            
            
    count = 0        
    last_counter = 0  
    new_count = 0
    
    
    while last_counter < numbers_of_args:
        
        values = [v for k, v in listDict.items()]
        next_one = values[last_counter][0]
       
        while new_count <= (numbers_of_args * 10):
            
            for k, v in (next_one):
                words_lists[count].append(v)
                new_count += 1

            break      
        count +=1
        last_counter +=1
        df = pd.DataFrame(list(zip(*words_lists[0:])), columns=args)
                             
    return df

nearest_neighbours("Penge", "Dårlig", "forskellige", "ubehagelig")

,Penge,Dårlig,forskellige,ubehagelig
0,anmelde,originale,",Lang",mangle
1,rydde,stjerner.,Stor,bankoverførsel.
2,ønskeliste,syningen,Der,fleece
3,webshops.Har,spammail,jævne,skrald
4,tider.,",Man",besværligt,dårligste
5,",Der",seks,fem,Jylland
6,leverer,montage.Det,med.,stiger
7,forsendelse!Har,pakkeservice,vask,rent
8,plejer,dårligt.,sædvanligt.,Enten
9,sandsynligt,placerer,rykkeren,kassen


In [ ]:
kmeans = KMeans(n_clusters=5)
data['cluster'] = kmeans.fit_predict(data['vec'].values.tolist())

In [ ]:
pca = PCA(n_components=3)
data['x'] = pca.fit_transform(data['vec'].values.tolist())[:,0]
data['y'] = pca.fit_transform(data['vec'].values.tolist())[:,1]
data['z'] = pca.fit_transform(data['vec'].values.tolist())[:,2]

scatter = plt.scatter(data['x'], data['y'], c=data['cluster'], cmap='rainbow', alpha=0.5)

# dc = data_cluster
# cc = cluster_colors
# cl = cluster_labels
print(data['cluster'].value_counts())
dc = data['cluster'].value_counts()
dc = -np.sort(-dc)
cc = data['cluster'].value_counts().index.tolist()

cl = []
for i in model.labels:
    cl.append(i.replace('__',''))

handles = scatter.legend_elements(num=cc)[0]  # extract the handles from the existing scatter plot

plt.legend(title='Ratings', handles=handles,  
           labels=[cl[0]+' '+str(dc[0]),
                   cl[1]+' '+str(dc[1]),
                   cl[2]+' '+str(dc[2]),
                   cl[3]+' '+str(dc[3]),
                   cl[4]+' '+str(dc[4])])

In [ ]:
fig = plt.figure(1, figsize=(10,10))
ax = Axes3D(fig, rect=[0, 0, 0.95, 1], elev=48, azim=134)
ax.scatter(data['x'],data['y'],data['z'], c=data['cluster'], cmap='rainbow', alpha=0.5)
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("z")
ax.set_facecolor('white')
plt.title("Review Clustering using K Means", fontsize=14)
plt.legend(title='Classes', handles=handles, 
           labels=[cl[0]+' '+str(dc[0]),
                   cl[1]+' '+str(dc[1]),
                   cl[2]+' '+str(dc[2]),
                   cl[3]+' '+str(dc[3]),
                   cl[4]+' '+str(dc[4])])